In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import keras
from keras.layers import Dense, Flatten
from keras.models import Model 
from keras.preprocessing.image import ImageDataGenerator, load_img , img_to_array


In [ ]:
tf.keras.applications.inception_v3.preprocess_input
from keras.applications.inception_v3 import preprocess_input

**STAGE 1**

*I built the base model using Kera's inception_v3 model*

In [ ]:
base_model=tf.keras.applications.inception_v3.InceptionV3(input_shape=(256,256,3),
    include_top=False
)

 This is a non-trainable model.
 
I am not training any layers, I will just use the outputs of the base model and add to our actual layer.

In [ ]:
 for layer in base_model.layers:
  layer.trainable= False

In [ ]:
X= Flatten()(base_model.output)
X= Dense(units=3, activation= 'sigmoid')(X)



I will now create our actual model using Adam's optimizer and binary_cross entropy as our loss function.

In [ ]:
# Final model
model=Model(base_model.input , X)

# compile model
model.compile(optimizer= 'adam',loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

#summary
model.summary()

In [ ]:
## Use Data generator to carry out our image processing
#My google drive which has already been uploaded to my colab directory has a leaf image folder containing the five different leave types in their seperate folders  

In [ ]:
train_datagen=ImageDataGenerator(featurewise_center= True, rotation_range=0.4, 
                                 width_shift_range=0.3, 
                                 horizontal_flip=True, 
                                 preprocessing_function= preprocess_input ,
                                  zoom_range=0.4, 
                                 shear_range=0.4)

train_data=train_datagen.flow_from_directory(directory="/content/drive/MyDrive/images2", 
                                             target_size=(256,256),
                                             batch_size=36)

In [ ]:
train_data.class_indices

In [ ]:
t_img , label= train_data.next()

In [ ]:
t_img.shape

In [ ]:
#tRAIN THE MODEL

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc= ModelCheckpoint(filepath="./team_model.h5", 
                    monitor="accuracy", 
                    verbose=1, 
                    save_best_only=True)

es= EarlyStopping(monitor= "accuracy",
                  min_delta= 0.01,
                  patience=5,
                  verbose=1)

cb=[mc,es] 

In [ ]:
his =model.fit_generator(train_data, 
                         steps_per_epoch=5, 
                         epochs=10, 
                         callbacks=cb)

In [ ]:
from keras.models import load_model

model= load_model("./team_model.h5")

In [ ]:
h= his.history
h.keys()

I plotted a chart of the loss and model accuracy

In [ ]:
plt.plot(h['loss'])
plt.plot(h['accuracy'])

plt.title('LOSS vs ACC')
plt.show()